### DNS Reflection/Amplification Detection and Mitigation

DNS reflection/amplification attacks are a special kind of DDoS attack that use DNS servers to flood a victim service or host with many DNS reply packets. In this assignment, you will learn about techniques to detect and mitigate this type of attack and will apply these techniques on a sample emulated network within mininet.

#### The Attack

In a DNS reflection/amplification attack, an attacker sends a DNS request to a DNS server with the source IP address being spoofed to represent the victim's IP address. Thus, the DNS server sends the reply to the victim. The attacker can use many compromised machines (botnets) to generate a very large number of such requests, which results in an overwhelming amount of traffic (DNS responses) headed towards the victim's machine. 

In addition to the reflection aspect, the attack also involves a concenpt known as "amplification", because the size of each DNS response is typically larger than the size of each corresponding request. As a result, even if the attacker launches the attack from a single compromised machine, the attack will have more overhead on the victim than the attacker. Of course, the power of the attack is much more significant when the attack is originated from many machines.

One place to read a bit more about reflection attacks in general is in the [BITAG report on SNMP reflection attacks](http://www.bitag.org/documents/SNMP-Reflected-Amplification-DDoS-Attack-Mitigation.pdf). Section 2.3 of the report details an example SNMP reflection attack, and much of the report discusses mitigation strategies. In this assignment, you'll focus on a _DNS_ reflection attack, which is a similar concept, albeit using the familiar DNS protocol instead of SNMP.

#### Detection

One way to detect DNS reflection is to keep track of the DNS requests and responses that each host sends and receives. Suppose there is a middlebox at the edge of the network that has sufficient resources to process DNS traffic as it passes through the network. 

This middlebox can inspect every outgoing DNS query; it can record, for each host, the identification number of that DNS query; it can also inspect every incoming DNS response. If the response's identification number does not match that of any of the requests recorded for the destination host, the middlebox will increment a counter. If the counter passes some threshold, the middlebox may determine that the host that is receiving unsolicited responses and is thus a victim of a reflection attack.

#### Mitigation

One way to mitigate DNS reflection is to rate-limit the attack traffic. More specifically, in the scenario described above, once the middlebox detects an attack towards a specific host, it can limit the rate of DNS responses that the host receives. One could think of completely blocking the traffic, but in the case of DNS, there could be some legitimate DNS responses that the host asked for and needs for maintaining its connection to the network. 

### This Assignment

In this assignment, you will detect and mitigate a DNS reflection/amplification attack on a sample network.

#### Sample Network

You can see a folder called *dev* in the same folder as this notebook. Inside *dev*, there is file called *start_net.py*. Running this file as root will setup the following network in mininet:
<img width=700 src="figures/network.png">
In this setup, **s1**, **h1**, and **h4** constitute your private network. **mb** is the middlebox sitting at the  edge; this middlebox can see all the incoming and outgoing traffic of your network. **h3** is a host that is running *bind*, an open DNS resolver, and **h2** is an attacker. 

Once you start the network, h1 and h4 will begin to send DNS requests to the DNS resolver every five seconds, and also start a separate ping trail to track their connection performance to the DNS resolver every two seconds. At the same time, h2 starts sending *spoofed* DNS requests on behalf of h1 to the DNS resolver every half a second. Thus, h1 is going to be the victim of a large volume of DNS response traffic that it has not asked for, namely a DNS reflection attack.

**Note:** the start_net.py script opens up a mininet CLI once the network is up and running. You can use mininet CLI commands to get network information and interact with the hosts and switches on your network if needed. To gracefully end the experiment, you should type *quit* on the mininet CLI. That takes care of cleaning up mininet and all the processes it has started. If exited with an exception, you should make sure to do "mn -c" as root, and also kill the processes that have "ping" or "dns" in their names (Use the same commands you used in the bufferbloat assignment).

#### Detection
The middlebox in the sample topology is implemented as a special kind of host in mininet. It runs a python script called *mb.py*, which you can find in the *dev* folder. This script uses a python library called *scapy* to sniff the packets from the two interfaces of the middlebox, applies a specific function to each packet, and sends it out of the correct interface to maintain the connectivity of the networks on the sides. You have to add some code in the designated area in this file to implement the detection and mitigation strategy introduced above.

More specifically, you have to check each packet that is going out of your network to see if it is a DNS request. If it is, you have to record a mapping from the source of the request to the request's identification number. Also, for each DNS response packet that is coming into your network, you have to make sure there has been a request for it from the destination host. You should keep track of a the number of unmatched responses for each host. If this number passes 200 for any host, the mitigation should be started for that host.

**Note:** The function that you have to change takes Python's scapy packet as input. Scapy is a powerful packet manipulation tool. We will cover its basics in precept, but it is your responsibility to learn the details. Feel free to take a look at [www.secdev.org/projects/scapy/](www.secdev.org/projects/scapy/) and [www.secdev.org/projects/scapy/doc/usage.html](www.secdev.org/projects/scapy/doc/usage.html) to learn more.

#### Mitigation

Once you detect a host as being attacked, you should use a tool called *tc* to rate limit the **DNS response traffic** sent **out** of the middlebox to **that specific host**. tc has a commandline program that is installed on linux by default. So once you detect the attack, call the tc command with appropriate arguments from the *mb.py* file to start rate limiting. Choose a reasonable tc mechanism with reasonable parameters, and **explain your choices in your readme**. Again, we will cover the basics of tc in the precept, but you have to make sure to learn the details from tutorials and documentations, such as the tc manpage or [lartc.org](http://lartc.org).

You should make sure that you are only rate limiting the attack traffic. Since you do not know which DNS responses are legitimate, you can rate limit all DNS responses to the victim host. The traffic of other hosts, or other types of traffic to the victom's host should not be rate limited. 

**Note:** make sure to initiate the mitigtaion only **once** after the threshold is passed, for each host. 

#### Testing

To make sure your detection and mitigation works correctly, you should complete a script that runs on h1 and h4. You can find this script called *test.py* in the *dev* folder. Using scapy, the script sniffs all the packets sent to the host on which the script is running. You should complete this script to keep track of the number of DNS responses and ping replies, and report them every minute. More specifically, the script should output, every minute, the number of DNS response and ping reply packets that it has seen in that minute. 

** Important note:** although in this specific setup, h1 is being the victim, we may (or may not!) test your assignment using h4 as victim. Therefore, your detection and mitigation implementation **should not** assume a specific host to be victim. Moreover, you **should not** have h1 (or h4's) specific details (such as ip address) or the rates at which the traffic is generated hardcoded in your code. 

### Extra Credit 

There is a technique to detect suspicious DNS requests and rate limit the response to them at the DNS resolver itself. The technique is called *Response Rate Limiting (RRL)*. bind, the DNS resolver running on h3, can be configured to use RRL. You can get extra credit up to 20% of the assignment grade by exploring RRL, how to set it up for bind, explaining them in your readme, and implement the whole detection and mitigation using RRL. Note that you only get the extra credit if you have a reasonable implementation for the mandatory part of the assignment. 